In [1]:
import dask.dataframe as dd

# JOB WORKLOADS

## Importing data

In [2]:
column_names_s17 = [
  "create_timestamp",
  "modify_timestamp",
  "job_id",
  "task_id",
  "instance_num",
  "status",
  "plan_cpu",
  "plan_mem"
]

column_types_s17 = {
  column_names_s17[0]: 'int64',
  column_names_s17[1]: 'int64',
  column_names_s17[2]: 'int32',
  column_names_s17[3]: 'int32',
  column_names_s17[4]: 'int64',
  column_names_s17[5]: 'string',
  column_names_s17[6]: 'float32',
  column_names_s17[7]: 'float64',
}

df_s17 = dd.read_csv('./s17_batch_task.csv', blocksize=25e8, names=column_types_s17, dtype=column_types_s17)

In [4]:

column_names_s18 = [
    "start_timestamp",
    "end_timestamp",
    "job_id",
    "task_id",
    "machineID",
    "status",
    "seq_no",
    "total_seq_no",
    "real_cpu_max",
    "real_cpu_avg",
    "real_mem_max",
    "real_mem_avg",
]

column_types_s18_i = {
    column_names_s18[0]: object,
    column_names_s18[1]: object,
    column_names_s18[2]: object,
    column_names_s18[3]: object,
    column_names_s18[4]: object,
    column_names_s18[5]: object,
    column_names_s18[6]: object,
    column_names_s18[7]: object,
    column_names_s18[8]: object,
    column_names_s18[9]: object,
    column_names_s18[10]: object,
    column_names_s18[11]: object,
}

df_s18_i = dd.read_csv('./s18_batch_instance.csv', blocksize=25e8, names=column_names_s18, dtype=column_types_s18_i)

column_types_s18 = {
    column_names_s18[0]: 'int64',
    column_names_s18[1]: 'int64',
    column_names_s18[2]: 'int32',
    column_names_s18[3]: 'int32',
    column_names_s18[4]: 'int32',
    column_names_s18[5]: object,
    column_names_s18[6]: 'int32',
    column_names_s18[7]: 'int32',
    column_names_s18[8]: 'float32',
    column_names_s18[9]: 'float32',
    column_names_s18[10]: 'float64',
    column_names_s18[11]: 'float64',
}

df_s18 = df_s18_i.fillna(-1).astype(column_types_s18)

In [ ]:
# df_s17.shape[0].compute()

# df_s17['job_id'].nunique().compute()

# df_s17.query('job_id == ').compute()

## Numbers in Brief

In [5]:
tasks_per_job = df_s17.groupby('job_id')['task_id'].count()

In [6]:
tasks_per_job_mean = tasks_per_job.mean().compute()
tasks_per_job_max = tasks_per_job.max().compute()
tasks_per_job_min = tasks_per_job.min().compute()

### Tasks per Job

In [7]:
tasks_per_job_mean, tasks_per_job_max, tasks_per_job_min

(6.219828584665277, 156, 1)

In [10]:
df_s17.query('task_id == 2').compute()

,create_timestamp,modify_timestamp,job_id,task_id,instance_num,status,plan_cpu,plan_mem
35656,2864,2943,2,2,15740,Terminated,50.0,0.007947


In [11]:
instances_per_task = df_s17.groupby('task_id')['instance_num'].sum()

In [15]:
instances_per_task_mean = instances_per_task.mean().compute()
instances_per_task_max = instances_per_task.max().compute()
instances_per_task_min = instances_per_task.min().compute()

### Instances per Task

In [26]:
instances_per_task_mean, instances_per_task_max, instances_per_task_min

(147.84453713704022, 64486, 1)

In [19]:
instances_per_job = df_s17.groupby('job_id')['instance_num'].sum()

In [20]:
instances_per_job_mean = instances_per_job.mean().compute()
instances_per_job_max = instances_per_job.max().compute()
instances_per_job_min = instances_per_job.min().compute()

## Instances per Job

In [21]:
instances_per_job_mean, instances_per_job_max, instances_per_job_min

(919.5676781715698, 66632, 1)

### Total Jobs in Alibaba 2017 data for job traces

In [22]:
no_of_jobs_in_alibaba_17 = df_s17['job_id'].nunique().compute()
no_of_jobs_in_alibaba_17

12951

In [ ]:
total_vms = df_s18['machineID'].unique().compute()

### Total no of VMs in Alibaba 2017 data for job traces

In [25]:
total_vms.count()

1294